# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [46]:
# Passo 1 - Entrar na internet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

navegador = webdriver.Firefox(
    executable_path=r'C:\Users\eduar\geckodriver.exe')

# Passo 2 - Pegar a cotação do dólar
#entrar no google
navegador.get("https://www.google.com/")

#pesquisar "cotação dólar"
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#pegar o número que aparece como resultado no google
time.sleep(10)
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)


# Passo 3 - Pegar a cotação do euro
#é o mesmo procedimento

navegador.get("https://www.google.com/")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

time.sleep(10)
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 4 - Pegar a cotação do ouro
#entrar no site do melhor câmbio
navegador.get('https://www.melhorcambio.com/ouro-hoje')

#pegar a cotação
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(',','.')
print(cotacao_ouro)







5.254099999999999
6.167525285
296.44


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [47]:
# Passo 5 - Importar e atualizar a base de dados

import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [48]:
# atualizar a cotação
    #aonde a coluna "Moeda" = Dólar
    #tabela.loc[linha, coluna]
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)

    # aonde a coluna "Moeda" = Euro
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)

    # aonde a coluna "Moeda" = Ouro
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)


# atualizar o Preço de Compra -> Preço Original * Cotação
tabela["Preço de Compra"] = tabela["Preço Base Original"] * tabela["Cotação"]
 
# atualizar o Preço de Venda -> Preço de Compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]
tabela["Preço de Venda"] = tabela["Preço de Venda"].map('{:.2f}'.format)

display(tabela)


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final,Preço de Compra,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.254100,4999.95,1.40,6999.930,5254.047459,7355.67
1,Carro Renault,4500.00,Euro,6.167525,27000.00,2.00,54000.000,27753.863783,55507.73
2,Notebook Dell,899.99,Dólar,5.254100,4499.95,1.70,7649.915,4728.637459,8038.68
3,IPhone,799.00,Dólar,5.254100,3995.00,1.70,6791.500,4198.025900,7136.64
4,Carro Fiat,3000.00,Euro,6.167525,18000.00,1.90,34200.000,18502.575855,35154.89
5,Celular Xiaomi,480.48,Dólar,5.254100,2402.40,2.00,4804.800,2524.489968,5048.98
6,Joia 20g,20.00,Ouro,296.440000,7000.00,1.15,8050.000,5928.800000,6818.12


### Agora vamos exportar a nova base de preços atualizada

In [49]:
# Passo 6 - Importar e atualizar a base de dados
tabela.to_excel("ProdutosAtualizado.xlsx", index=False)